In [62]:

import numpy as np
import torch
import glob
import cv2
import os
import os.path as osp
import pickle
import json
import pandas as pd

In [ ]:
root = "C:/Users/User/source/CuTi/matterport/rgb/"

In [ ]:
print(root)

In [ ]:
path = "matterport/mp3d_planercnn_json/cached_set_train.json"

In [ ]:
with open(osp.join(path)) as f:
   split = json.load(f)

In [ ]:
(split['data'][0]['0']['file_name']).split('/')[6:]

In [ ]:
len(split['data'])


In [ ]:
img_name = os.path.join(root,'/'.join(str(split['data'][0]['0']['file_name']).split('/')[6:]))
img_name

In [ ]:
images = []
for i in range(len(split['data'])):
    
    for imgnum in ['0', '1']:
        img_name = os.path.join(root, '/'.join(str(split['data'][i][imgnum]['file_name']).split('/')[6:]))
        images.append(img_name)

In [ ]:
len(images)

In [ ]:
split['data'][0]['rel_pose']['position']+split['data'][i]['rel_pose']['rotation']

In [ ]:
scene_info = {'images': [], 'poses': [], 'intrinsics': []}
base_pose = np.array([0,0,0,0,0,0,1])

In [ ]:
for i in range(len(split['data'])):
    images = []
    for imgnum in ['0', '1']:
        img_name = os.path.join(root, '/'.join(str(split['data'][i][imgnum]['file_name']).split('/')[6:]))
        images.append(img_name)
    
    rel_pose = np.array(split['data'][i]['rel_pose']['position'] + split['data'][i]['rel_pose']['rotation'])
    og_rel_pose = np.copy(rel_pose)

    # on matterport, we scale depths to balance rot & trans loss
    rel_pose[:3] /= 5.0
    cprp = np.copy(rel_pose)
    rel_pose[6] = cprp[3] # swap 3 & 6, we want W last for consistency with our other datasets
    rel_pose[3] = cprp[6]
    if rel_pose[6] < 0: # normalize quaternions to have positive "W"
        rel_pose[3:] *= -1
    poses = np.vstack([base_pose, rel_pose])

    intrinsics = np.array([[517.97, 517.97, 320, 240], [517.97, 517.97, 320, 240]]) # 480 x 640 imgs

    scene_info['images'].append(images)
    scene_info['poses'] += [poses]
    scene_info['intrinsics'] += [intrinsics]

    print(scene_info)

In [74]:
images_list = ["matterport/rgb/rgb/1LXtFkjw3qL/0_2_0.png","matterport/rgb/rgb/1LXtFkjw3qL/0_2_1.png"]

In [75]:
images = []

In [77]:
for i in range(2):
    images.append(cv2.imread(images_list[i]))

In [78]:
images

[array([[[ 71, 107,  95],
         [ 72, 108,  96],
         [ 71, 110,  95],
         ...,
         [143, 139, 134],
         [143, 139, 134],
         [143, 139, 134]],
 
        [[ 71, 107,  96],
         [ 72, 108,  96],
         [ 72, 108,  96],
         ...,
         [143, 139, 134],
         [143, 139, 134],
         [143, 139, 134]],
 
        [[ 72, 105,  97],
         [ 71, 106,  95],
         [ 71, 105,  95],
         ...,
         [143, 139, 134],
         [143, 139, 134],
         [143, 139, 134]],
 
        ...,
 
        [[157, 174, 161],
         [155, 172, 159],
         [152, 167, 153],
         ...,
         [153, 153, 147],
         [153, 153, 147],
         [153, 153, 147]],
 
        [[159, 173, 161],
         [164, 178, 165],
         [159, 173, 160],
         ...,
         [153, 153, 147],
         [153, 153, 147],
         [153, 153, 147]],
 
        [[162, 176, 164],
         [184, 197, 183],
         [138, 150, 152],
         ...,
         [153, 153, 147],
  

In [79]:
images = np.stack(images).astype(np.float32)

In [80]:
images

array([[[[ 71., 107.,  95.],
         [ 72., 108.,  96.],
         [ 71., 110.,  95.],
         ...,
         [143., 139., 134.],
         [143., 139., 134.],
         [143., 139., 134.]],

        [[ 71., 107.,  96.],
         [ 72., 108.,  96.],
         [ 72., 108.,  96.],
         ...,
         [143., 139., 134.],
         [143., 139., 134.],
         [143., 139., 134.]],

        [[ 72., 105.,  97.],
         [ 71., 106.,  95.],
         [ 71., 105.,  95.],
         ...,
         [143., 139., 134.],
         [143., 139., 134.],
         [143., 139., 134.]],

        ...,

        [[157., 174., 161.],
         [155., 172., 159.],
         [152., 167., 153.],
         ...,
         [153., 153., 147.],
         [153., 153., 147.],
         [153., 153., 147.]],

        [[159., 173., 161.],
         [164., 178., 165.],
         [159., 173., 160.],
         ...,
         [153., 153., 147.],
         [153., 153., 147.],
         [153., 153., 147.]],

        [[162., 176., 164.],
       

In [81]:
images = torch.from_numpy(images).float()


In [82]:
images.shape

torch.Size([2, 480, 640, 3])

In [83]:
images = images.permute(0, 3, 1, 2)

In [84]:
images.shape

torch.Size([2, 3, 480, 640])

In [92]:
import torch
import torchvision.transforms as transforms
import numpy as np
import torch.nn.functional as F
def color_transform( images):
    """color jittering"""
    num, ch, ht, wd = images.shape
    images = images.permute(1, 2, 3, 0).reshape(ch, ht, wd * num)
    images = 255 * augcolor(images[[2, 1, 0]] / 255.0)
    return (
        images[[2, 1, 0]].reshape(ch, ht, wd, num).permute(3, 0, 1, 2).contiguous()
    )

augcolor = transforms.Compose(
            [
                transforms.ToPILImage(),
                transforms.ColorJitter(
                    brightness=0.25, contrast=0.25, saturation=0.25, hue=0.4 / 3.14
                ),
                transforms.RandomGrayscale(p=0.1),
                transforms.ToTensor(),
            ]
        )

In [93]:
images = color_transform(images)

In [94]:
images

tensor([[[[ 57.,  59.,  56.,  ..., 165., 165., 165.],
          [ 57.,  59.,  59.,  ..., 165., 165., 165.],
          [ 59.,  57.,  57.,  ..., 165., 165., 165.],
          ...,
          [180., 176., 175.,  ..., 177., 177., 177.],
          [183., 189., 184.,  ..., 177., 177., 177.],
          [187., 219., 153.,  ..., 177., 177., 177.]],

         [[116., 117., 120.,  ..., 157., 157., 157.],
          [116., 117., 117.,  ..., 157., 157., 157.],
          [114., 115., 114.,  ..., 157., 157., 157.],
          ...,
          [207., 205., 198.,  ..., 176., 176., 176.],
          [206., 213., 206.,  ..., 176., 176., 176.],
          [211., 240., 176.,  ..., 176., 176., 176.]],

         [[ 92.,  93.,  91.,  ..., 150., 150., 150.],
          [ 93.,  93.,  93.,  ..., 150., 150., 150.],
          [ 95.,  92.,  92.,  ..., 150., 150., 150.],
          ...,
          [183., 181., 175.,  ..., 168., 168., 168.],
          [184., 189., 183.,  ..., 168., 168., 168.],
          [188., 217., 176.,  ...

In [96]:
images.shape

torch.Size([2, 3, 480, 640])

In [97]:
sizey,sizex = [384, 512]

In [100]:
scalex = sizex / images.shape[-1]
scalex

0.8

In [101]:
scaley = sizey / images.shape[-2]
scaley

0.8

In [1]:
import torch

In [8]:
x = torch.randn(6,6,128,8,8)

In [9]:
y = torch.randn(6,6,128,8,8)

In [10]:
z = x @ y

In [11]:
z.shape

torch.Size([6, 6, 128, 8, 8])